In [ ]:
# Plot Sizes

In [1]:
from taxonomy_time_machine.models import Taxonomy

from glob import glob
import os
import polars as pl
import altair as alt

In [10]:
DEFAULT_PALETTES = {
    "ocx": [
        "#16347B",
        "#0072C7",
        "#01ACEC",
        "#97E9FC",
        "#0A605E",
        "#1DA893",
        "#3DD8BE",
        "#ABEFE2",
        "#37257D",
        "#9C78E0",
        "#CBC0F9",
        "#E3DDFF",
        "#BC5B00",
        "#EB984A",
        "#FCE34D",
        "#FEF2A3",
        "#950303",
        "#DD3A3A",
        "#FF8D8B",
        "#FFD5CB",
        "#771354",
        "#C13A8B",
        "#F28BBF",
        "#F9D9E7",
    ],
    "ocx_categorical": [
        "#0072C7",
        "#1DA893",
        "#9C78E0",
        "#EB984A",
        "#DD3A3A",
        "#C13A8B",
        "#01ACEC",
        "#3DD8BE",
        "#CBC0F9",
        "#FCE34D",
        "#FF8D8B",
        "#F28BBF",
        "#16347B",
        "#0A605E",
        "#37257D",
        "#BC5B00",
        "#950303",
        "#771354",
    ],
    "tableau10": [
        "#4e79a7",
        "#f28e2b",
        "#e15759",
        "#76b7b2",
        "#59a14f",
        "#edc948",
        "#b07aa1",
        "#ff9da7",
        "#9c755f",
        "#bab0ac",
    ],
}

OCX_DARK_GREEN = "#128887"


In [14]:

def onecodex_theme():
    onecodex_palette = ["#ffffcc", "#c7e9b4", "#7fcdbb", "#41b6c4", "#2c7fb8", "#264153"]

    font_family = "Fira Sans, Helvetica"

    return {
        "config": {
            "range": {
                "heatmap": list(reversed(onecodex_palette)),
                "category": DEFAULT_PALETTES["ocx_categorical"],
                "ramp": list(reversed(onecodex_palette)),
            },
            "area": {"fill": OCX_DARK_GREEN},
            "bar": {"fill": OCX_DARK_GREEN},
            "mark": {"color": OCX_DARK_GREEN},
            "axis": {
                "labelFont": font_family,
                "labelFontSize": 12,
                "titleFont": font_family,
                "titleFontSize": 12,
                "grid": True,
            },
            "legend": {
                "labelFont": font_family,
                "labelFontSize": 12,
                "titleFont": font_family,
                "titleFontSize": 12,
            },
            "title": {"font": font_family},
            "view": {"width": 400, "height": 400, "strokeWidth": 0},
            "background": "white",
        }
    }


In [15]:
alt.themes.register("onecodex", onecodex_theme)
alt.themes.enable("onecodex")


ThemeRegistry.enable('onecodex')

In [2]:
tab = []
for db_path in glob("archive-*.db"):
    print(db_path)
    deduplicated = "deduplicated" in db_path
    n_archives = int(db_path.split("-")[1].split(".")[0])
    ttm = Taxonomy(db_path)

    res = dict(
        list(
            ttm.cursor.execute(
                "select count(distinct tax_id) as unique_tax_ids, count(*) as n_rows from taxonomy;"
            )
        )[0]
    )

    tab.append(
        {
            **res,
            "db_size": os.stat(db_path).st_size,  # bytes
            "n_archives": n_archives,
            "is_deduplicated": "Deduplicated" if deduplicated else "Naive",
            "path": db_path,
        }
    )

archive-20-deduplicated.db
archive-20-full.db
archive-50-full.db
archive-1-deduplicated.db
archive-5-full.db
archive-10-full.db
archive-1-full.db
archive-5-deduplicated.db
archive-10-deduplicated.db
archive-2-full.db
archive-2-deduplicated.db
archive-50-deduplicated.db


In [41]:
alt.Chart(
    pl.DataFrame(tab)
    .with_columns((pl.col("db_size") / (1024**3)).alias("db_size_GiB"))
    .to_pandas()
).mark_line(point=True).encode(
    x=alt.X("unique_tax_ids", title="Total Taxa"),
    y=alt.Y("db_size_GiB", scale=alt.Scale(type="linear"), title="DB Size (GiB)"),
    color=alt.Color("is_deduplicated", title="Mode"),
).properties(title="Database Size vs. Implementation").properties(width=530, height=420)

/Users/audy/Code/science-vessel/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/audy/Code/science-vessel/.venv/lib/python3.12/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)